Using the following dataset : http://archive.ics.uci.edu/dataset/19/car+evaluation

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

### Importing

In [2]:
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data"
columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
df = pd.read_csv(url, names=columns)

In [3]:
df.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


### Data Preprocessing

In [4]:
df_encoded = pd.get_dummies(df, columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])

X = df_encoded.drop('class', axis=1)
y = df_encoded['class']

encoder = LabelEncoder()
y_trans = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_trans, test_size=0.2, random_state=42)

1. Apply XGBoost classifier on the dataset using the following parameter : Objective = Multi Softmax , print out the classification report

In [5]:
xgb_classifier = XGBClassifier(objective='multi:softmax')
xgb_classifier.fit(X_train, y_train)
y_pred_xgb = xgb_classifier.predict(X_test)

print("Classification Report: ")
print(classification_report(y_test, y_pred_xgb))

Classification Report: 
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        83
           1       0.61      1.00      0.76        11
           2       1.00      1.00      1.00       235
           3       1.00      1.00      1.00        17

    accuracy                           0.98       346
   macro avg       0.90      0.98      0.93       346
weighted avg       0.99      0.98      0.98       346



2. Apply Catboost classifier on the dataset using the following parameter : Iterations = 150 , depth = 5, learning_rate = 0.3 , loss_function='MultiClass' 
3. Also set the verbose for the Catboost model fit to 15

In [6]:
cat_classifier = CatBoostClassifier(iterations=150, depth=5, learning_rate=0.3, loss_function='MultiClass', verbose=15)
cat_classifier.fit(X_train, y_train)
y_pred_cat = cat_classifier.predict(X_test)

print("Classification Report: ")
print(classification_report(y_test, y_pred_cat))

0:	learn: 0.9474975	total: 136ms	remaining: 20.3s
15:	learn: 0.2026547	total: 193ms	remaining: 1.62s
30:	learn: 0.1136741	total: 229ms	remaining: 878ms
45:	learn: 0.0760837	total: 262ms	remaining: 591ms
60:	learn: 0.0548474	total: 295ms	remaining: 430ms
75:	learn: 0.0425822	total: 330ms	remaining: 321ms
90:	learn: 0.0346446	total: 367ms	remaining: 238ms
105:	learn: 0.0287692	total: 401ms	remaining: 166ms
120:	learn: 0.0247338	total: 434ms	remaining: 104ms
135:	learn: 0.0219709	total: 466ms	remaining: 47.9ms
149:	learn: 0.0195640	total: 497ms	remaining: 0us
Classification Report: 
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        83
           1       0.59      0.91      0.71        11
           2       1.00      1.00      1.00       235
           3       0.94      1.00      0.97        17

    accuracy                           0.98       346
   macro avg       0.88      0.96      0.91       346
weighted avg       0.98      0.98

3. Compare the Overall Accuracy and the individual accuracy ratings between the two classifiers 

In [7]:
accuracy_xgb = (y_test == y_pred_xgb).mean()
accuracy_cat = (y_test == y_pred_cat).mean()

print(f"Overall accuracy")
print(f"XGBoost: {accuracy_xgb}")
print(f"CatBoost: {accuracy_cat}")

Overall accuracy
XGBoost: 0.9797687861271677
CatBoost: 0.518109525877911


XGBoost has a definitely higher accuracy than CatBoost.